In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [4]:
df = ssql("""
SELECT t1.date,
       t1.distinct_id,
       t2.game_level
FROM
  (SELECT date,distinct_id,
               match_id
   FROM events
   WHERE event = 'gameStart'
     AND game_played= 5
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-30' AND current_date()) t1
LEFT JOIN
  (SELECT date,distinct_id,
               game_level,
               match_id
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-11-30' AND current_date())t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
AND t1.match_id = t2.match_id
""")

In [5]:
df

,date,distinct_id,game_level
0,2020-12-10,135401432,3.0
1,2020-12-10,134232602,2.0
2,2020-12-09,146786005,3.0
3,2020-12-09,122024635,3.0
4,2020-12-09,27542925,2.0
...,...,...,...
448647,2020-12-10,20585802,NaN
448648,2020-12-01,119687008,NaN
448649,2020-12-02,131317632,NaN
448650,2020-12-08,24658375,NaN


In [9]:
df_group = df.groupby(['date','game_level']).count()
df_group

distinct_id
date       game_level             
2020-11-30 0.0                8354
           1.0                1555
           2.0                9279
           3.0               16683
           4.0                4003
           5.0                   9
2020-12-01 1.0                1742
           2.0                9862
           3.0               18528
           4.0                5259
           5.0                  11
2020-12-02 1.0                1613
           2.0                9006
           3.0               18155
           4.0                5202
           5.0                   9
2020-12-03 1.0                1260
           2.0                7981
           3.0               17165
           4.0                5299
           5.0                   8
2020-12-04 1.0                2118
           2.0               11268
           3.0               21254
           4.0                5776
           5.0                   8
2020-12-05 1.0                5574
           2.0               28862
           3.0               40275
           4.0                8726
           5.0                  14
2020-12-06 1.0                5318
           2.0               27508
           3.0               41673
           4.0                9075
           5.0                   9
2020-12-07 1.0                1289
           2.0                7969
           3.0               17296
           4.0                5307
           5.0                   8
2020-12-08 1.0                1111
           2.0                6712
           3.0               14274
           4.0                4435
           5.0                   6
2020-12-09 1.0                 933
           2.0                6060
           3.0               13380
           4.0                4052
           5.0                   7
2020-12-10 1.0                 513
           2.0                3292
           3.0                8502
           4.0                2968
           5.0                   4
           6.0                   1

In [10]:
df_group.to_clipboard()